In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Config

In [ ]:
class CFG:
    device = "cuda"
    model_name = "tf_efficientnet_b0_ns"
    image_size = 224 #528でpretrain"https://github.com/rwightman/pytorch-image-models/blob/master/results/results-imagenet.csv"
    batch_size = 30
    num_workers = 1
    num_epochs = 10
    fold = 0
    seed = 123   

In [ ]:
!pip install timm

乱数固定

In [ ]:
import os
import gc
import cv2
import math
import copy
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import random
import numpy as np
import torch

def torch_fix_seed (seed):
    #Python random
    random.seed(seed)
    #Numpy
    np.random.seed(seed)
    #pyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    #https://qiita.com/north_redwing/items/1e153139125d37829d2d
    torch.backends.cudnn.benchmark = False 
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algoristhms = True
    
torch_fix_seed (CFG.seed)

データの確認

In [ ]:
from glob import glob

#指定のコンペから開いたnotebookじゃないと、ダメ
root_dir = "../input/happy-whale-and-dolphin"
#globは、指定されたパターンに一致するパスを取得
#glob中の*はワールドカードを表す
#print中の*は、リストのアンパックを表す、要素ごとにとりだすが、今回はsepで要素間が改行
print(*glob(f"{root_dir}/*"), sep="\n")
image_dir = f"{root_dir}/train_images"

In [ ]:
import pandas as pd

train = pd.read_csv(f"{root_dir}/train.csv")

train.head()

画像読み込み

In [ ]:
import cv2
import matplotlib.pyplot as plt

#一番上の画像をパスで取得
image = cv2.imread(f"{image_dir}/{train['image'].values[0]}")
#RGBへの変換
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.title(train["species"].values[0])
plt.imshow(image)
plt.show

datasetの定義

In [ ]:
from torch.utils.data import Dataset

#Datasetを継承して、学習用データセットのクラスをつくる
class WhaleDataset(Dataset):
    def __init__(self, pathes, classes, transform = None):
        self.pathes = pathes
        self.classes = classes
        self.transform = transform
    def __len__(self):
        return len(self.pathes)
    #WhaleDataset(dataset)(index)となり、indexに対応した、cv2.imreadで処理されたデータおよび、classが出てくる
    def __getitem__(self, index):
        image = cv2.imread(f"{image_dir}/{self.pathes[index]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = transform(image=image)["image"]
        return image, torch.tensor(self.classes[index]).long()

In [ ]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
#LEのfitとtransformを一度に行う便利な関数。カテゴリ変数を数値化(0~)
train["le_individual_id"] = LE.fit_transform(train["individual_id"])

#LEにはclasses_が定義されている。_は名前の衝突を防ぐために使う
print(len(LE.classes_))
print(LE.classes_)
print(train["le_individual_id"])
dataset = WhaleDataset(train["image"].values, train["le_individual_id"].values)

image, label = dataset[0]
plt.imshow(image)
plt.title(label)
plt.show()

In [ ]:
skf = StratifiedKFold(n_splits=5)

#層状K分割だから、全体のデータXが必要？
for fold, ( _, val_) in enumerate(skf.split(X=train, y=train.individual_id)):
    train.loc[val_ , "kfold"] = fold
    print(_)
    print(val_)
        

In [ ]:
train

In [ ]:
train["image"].values

In [ ]:
dataset[1]

In [ ]:
train.head()

画像前処理の定義
Resizeで画像サイズを処理、Nomalizeで色情報を変換、ToTensorV2でtensorに変換

In [ ]:
from albumentations import Compose, Resize, Normalize
from albumentations.pytorch import ToTensorV2

#datasetのうち後半はひつようないから_で済ましている
image, _ = dataset[0]

#例
transform = Resize(100,100)
transformed = transform(image=image)["image"]
plt.imshow(transformed)
plt.title("Resize")
plt.show()

transform = Compose([Resize(100,100), Normalize()])
transformed = transform(image=image)["image"]
plt.imshow(transformed)
plt.title("Resize and Normalize")
plt.show()

In [ ]:
transform = Compose([Resize(CFG.image_size, CFG.image_size),
                    Normalize(),
                    ToTensorV2()])
dataset = WhaleDataset(train["image"].values, train["le_individual_id"].values, transform)

image, num = dataset[1]
print(image)
print(num)

DataLoaderに載せる

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, num_workers = CFG.num_workers, batch_size = CFG.batch_size, shuffle = True, drop_last = True, pin_memory = True)

timmを用いてモデルを定義

学習

In [ ]:
import timm

#クラス情報はLEに入っている
model = timm.create_model(CFG.model_name, num_classes = len(LE.classes_))
model.to(CFG.device)
model

In [ ]:
import torch.nn as nn
import torch.optim as optim

#画像分類ではクロスエントロピーが一般的
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)

Cross validation

In [ ]:
from sklearn.model_selection import KFold

# とりあえず5-foldで分ける
kf = KFold(n_splits=5, shuffle=True, random_state=CFG.seed)
fold = np.zeros(len(train), dtype=np.int32)

for i, (train_idx, valid_idx) in enumerate(kf.split(train["image"])):
    fold[valid_idx] = i
    print(i)
    print(train_idx)
    print(valid_idx)
train["fold"] = fold

In [ ]:
tr = train[train["fold"]!=CFG.fold]
vl = train[train["fold"]==CFG.fold]
train_dataset = WhaleDataset(tr["image"].values, LE.transform(tr["individual_id"]), transform)
valid_dataset = WhaleDataset(vl["image"].values, LE.transform(vl["individual_id"]), transform)
train_dataloader = DataLoader(train_dataset, num_workers=CFG.num_workers, batch_size=CFG.batch_size,
                              shuffle=True, drop_last=True, pin_memory=True)
# validation用ではshuffleやdrop_lastをFalseにする
valid_dataloader = DataLoader(valid_dataset, num_workers=CFG.num_workers, batch_size=CFG.batch_size,
                              shuffle=False, drop_last=False, pin_memory=True)

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math


class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, 
                 m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CONFIG['device'])
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output


In [ ]:
 metric = ArcMarginProduct(512, 15587, s=30, m=0.05, easy_margin=True)

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

model = timm.create_model(CFG.model_name, num_classes=len(LE.classes_))
model.to(CFG.device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_losses = []
valid_losses = []
accs = []

for epoch in range(1, CFG.num_epochs+1):
    print(f"Epoch : {epoch}")
    count = 0
    loss_sum = 0
    model.train()
    for inputs, labels in tqdm(train_dataloader):
        bs, _, _, _ = inputs.size()
        optimizer.zero_grad()
        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)
        features = model(inputs)
        outputs = metrics.forward(features, labels)
        loss = criterion(outputs, labels)
        loss_sum += loss.item()*bs
        count += bs
        loss.backward()
        optimizer.step()
    print(f"Train Loss {loss_sum/count:.4f}")
    train_losses.append(loss_sum/count)

    count = 0
    loss_sum = 0
    # validation用ではeval関数によってmodelを推論モードにする必要がある
    model.eval()
    gt = []
    preds = []
    # validationでは誤差逆伝播を行わないので、勾配情報が不要
    # このtorch.no_gradを使うと勾配情報を計算しないようになり、計算時間を節約できる
    with torch.no_grad():
        for inputs, labels in tqdm(valid_dataloader):
            bs, _, _, _ = inputs.size()
            inputs = inputs.to(CFG.device)
            labels = labels.to(CFG.device)
            features = model(inputs)
            outputs = metrics.forward(features, labels)
            gt.append(labels.to("cpu").numpy())
            preds.append(F.softmax(outputs, dim=1).to("cpu").numpy())
            loss = criterion(outputs, labels)
            loss_sum += loss.item()*bs
            count += bs
    gt = np.concatenate(gt)
    preds = np.concatenate(preds)
    acc = accuracy_score(gt, np.argmax(preds, axis=1))
    print(f"Valid Loss {loss_sum/count:.4f}")
    valid_losses.append(loss_sum/count)
    print(f"Accuracy {acc:.4f}")
    accs.append(acc)

In [ ]:
models

In [ ]:
plt.plot(range(1, CFG.num_epochs+1), train_losses, label="train")
plt.plot(range(1, CFG.num_epochs+1), valid_losses, label="valid")
plt.legend()
plt.title("Cross Entropy Loss")
plt.xlabel("Epoch")
plt.ylabel("CE")
plt.show()

plt.plot(range(1, CFG.num_epochs+1), accs)
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("ACC")
plt.ylim(0, 1)
plt.show()

In [ ]:
LE.inverse_transform(np.argmax(preds, axis=1))

In [ ]:
LE.inverse_transform(gt)

In [ ]:
# nunpy.nparrayにおける[:, 0]は、0列目の行列を表す。
vl["prediction"] = LE.inverse_transform(np.argmax(preds, axis=1))
for i, c in enumerate(LE.classes_):
    vl[f"pred_{c}"] = preds[:, i]

In [ ]:
for i, c in enumerate(LE.classes_):
    print(i, c)

In [ ]:
vl

In [ ]:
_, top_idx = torch.topk(torch.tensor(preds), k=5)
#nはつまり、行数
n = top_idx.size()[0]
#LE.inverse_transformは、一行のnparrayが入力される。だからreshape(n*5)
top_species = LE.inverse_transform(top_idx.numpy().reshape(n*5)).reshape(n, 5)

for i in range(5):
    vl[f"top {i+1}"] = top_species[:, i]

In [ ]:
vl

In [ ]:
model = timm.create_model(CFG.model_name, num_classes=len(LE.classes_))
print(model)